In [18]:
#
# _____ ______  _______      _  _  _ _______ _______ _______  _____  __   _      _     _
#   |   |_____] |  |  |      |  |  | |_____|    |    |______ |     | | \  |       \___/ 
# __|__ |_____] |  |  |      |__|__| |     |    |    ______| |_____| |  \_|      _/   \_                                                                                       
#

In [19]:
!pip install "tracing_auto_instrumentation[ibm]"

In [1]:
import dotenv
import logging
import os

from enum import Enum

from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

from lastmile_eval.rag.debugger.api import LastMileTracer
from lastmile_eval.rag.debugger.tracing.sdk import get_lastmile_tracer

from tracing_auto_instrumentation.ibm import wrap_watson

/Users/b7r6/Projects/tracing_auto_instrumentation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
logger = logging.getLogger(__name__)

In [3]:
class Mode(Enum):
    GENERATE = "GENERATE"
    GENERATE_TEXT = "GENERATE_TEXT"


def init_watson_model() -> Model:
    # To display example params enter
    GenParams().get_example_values()
    generate_params = {GenParams.MAX_NEW_TOKENS: 25}

    watson_model = Model(
        model_id=ModelTypes.GRANITE_13B_CHAT_V2,
        params=generate_params,
        credentials=dict(
            api_key=os.getenv("WATSONX_API_KEY"),
            url="https://us-south.ml.cloud.ibm.com",
        ),
        space_id=os.getenv("WATSONX_SPACE_ID"),
        verify=None,
        validate=True,
    )

    return watson_model

In [9]:
def run_generate(prompt: str, trace_name: str) -> None:
    tracer: LastMileTracer = get_lastmile_tracer(
        trace_name, os.getenv("LASTMILE_API_TOKEN")
    )

    watson_model: Model = init_watson_model()

    tracer.log("start lastmile wrap...")
    wrapper = wrap_watson(watson_model, tracer)
    tracer.log("lastmile wrap complete.")


    with tracer.start_as_current_span(trace_name) as span:
        tracer.log("start watsonx generate...")
        response = wrapper.generate(prompt)
        tracer.log(f"watsonx generate: {response=}")

In [10]:
def run_generate_text(prompt: str, trace_name: str) -> None:
    tracer: LastMileTracer = get_lastmile_tracer(
        trace_name, os.getenv("LASTMILE_API_TOKEN")
    )

    watson_model: Model = init_watson_model()

    tracer.log("start lastmile wrap...")
    wrapper = wrap_watson(watson_model, tracer)
    tracer.log("lastmile wrap complete.")

    with tracer.start_as_current_span(trace_name) as span:
        tracer.log("start watsonx generate_text...")
        response = wrapper.generate_text(prompt)
        tracer.log(f"watsonx generate_text: {response=}")

In [11]:
def run_generation(mode: Mode, prompt: str, trace_name: str) -> None:
    logger.info("IBM WatsonX Generation Script Starting...")

    # n.b. required for multiple api keys, make sure `.env` is in your local path
    dotenv.load_dotenv()

    if Mode.GENERATE == mode:
        logger.info(f"running with mode: {mode}")
        run_generate(prompt, trace_name)
    elif Mode.GENERATE_TEXT == mode:
        logger.info(f"running with mode: {mode}")
        run_generate_text(prompt, trace_name)
    else:
        logger.error(f"unsupported mode: {mode}")

In [12]:
run_generation(
    mode=Mode.GENERATE,
    prompt="Can you please tell me an amusing anecdote about Thomas Edison?",
    trace_name="elementary-my-dear-watson",
)

2024-05-22 12:41:39,459 - IBM WatsonX Generation Script Starting...
2024-05-22 12:41:39,469 - running with mode: Mode.GENERATE
2024-05-22 12:41:39,531 - Overriding of current TracerProvider is not allowed
2024-05-22 12:41:40,356 - Client successfully initialized
2024-05-22 12:41:41,157 - Successfully finished Get available foundation models for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-05-10&space_id=c6dd344f-1aab-4b5e-8edb-2a21fa1e0de0&limit=200'
2024-05-22 12:41:41,157 - Response(GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-05-10&space_id=c6dd344f-1aab-4b5e-8edb-2a21fa1e0de0&limit=200): {"total_count":25,"limit":200,"first":{"href":"https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-05-10&space_id=c6dd344f-1aab-4b5e-8edb-2a21fa1e0de0&limit=200"},"resources":[{"model_id":"baai/bge-large-en-v1","label":"bge-large-en-v1","provider":"baai","source":"baai","functions":[{"id":"embedding"}],"